In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../data/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../data/sideeffect.csv');

In [3]:
-- SELECT * FROM Drug;
-- SELECT * FROM Pathology;
-- SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [ ]:
DROP VIEW interaction30mais IF EXISTS;
DROP VIEW interaction IF EXISTS;

CREATE VIEW interaction AS
SELECT A.codeDrug as drugA, b.codeDrug as drugB
FROM drugUse AS A, drugUse AS B
WHERE A.idPerson = B.idPerson and A.codeDrug < B.codeDrug;
SELECT * from interaction;

CREATE VIEW interaction30mais AS
SELECT drugA, drugB, count(*) weight
from interaction
group by drugA,drugB
having weight>30;

SELECT * from interaction30mais;

java.sql.SQLException:  

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [4]:
DROP TABLE DrugInteraction IF EXISTS;
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../data/drug-interaction.csv');

In [5]:
DROP VIEW DrugInteraction30 IF EXISTS;
CREATE VIEW DrugInteraction30 AS SELECT * FROM DrugInteraction WHERE weight > 30 ORDER BY druga, drugb;
SELECT * FROM DrugInteraction30;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [6]:
SELECT druga, drugb, weight, CAST(weight AS DOUBLE)/(SELECT count(*) FROM DrugInteraction) as suporte
FROM DrugInteraction30
ORDER By suporte

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [7]:
SELECT DrugInteraction30.druga, CAST(count(*) AS DOUBLE)/(SELECT count(*) FROM DrugInteraction) as confianca
FROM DrugInteraction,DrugInteraction30
WHERE DrugInteraction.druga = DrugInteraction30.druga
GROUP BY DrugInteraction.druga
ORDER BY confianca

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [42]:
DROP VIEW CombinedSideEffectFrequency IF EXISTS;
DROP VIEW NumberOfSideEffectsByDrugCombination IF EXISTS;
DROP VIEW NumberOfUsesByDrugCombination IF EXISTS;
DROP VIEW AloneSideEffectFrequency IF EXISTS;
DROP VIEW NumberOfSideEffectsByDrug IF EXISTS;
DROP VIEW NumberOfUsesByDrug IF EXISTS;
DROP VIEW DrugIntersection IF EXISTS;
DROP VIEW DrugAlone IF EXISTS;
DROP VIEW DrugsPerPerson IF EXISTS;
DROP VIEW ReducedDrugUse IF EXISTS;

CREATE VIEW ReducedDrugUse AS
SELECT *
FROM DrugUse
ORDER BY idPerson
LIMIT 300;

CREATE VIEW DrugsPerPerson AS
SELECT idPerson,count(codeDrug) as drugCount
FROM ReducedDrugUse
GROUP BY idPerson;

CREATE VIEW DrugAlone AS
SELECT ReducedDrugUse.idPerson, ReducedDrugUse.codeDrug
FROM ReducedDrugUse, DrugsPerPerson
WHERE ReducedDrugUse.idPerson = DrugsPerPerson.idPerson AND DrugsPerPerson.drugCount = 1;

CREATE VIEW DrugIntersection AS
SELECT Du1.idPerson, Du1.codeDrug as druga, Du2.codeDrug AS drugb
FROM ReducedDrugUse AS DU1, ReducedDrugUse AS DU2
WHERE DU1.idPerson = DU2.idPerson AND Du1.codeDrug <> Du2.codeDrug AND DU1.codeDrug < DU2.codeDrug

In [43]:
CREATE VIEW NumberOfSideEffectsByDrug AS
SELECT codeDrug,count(codeDrug) as sideeffects
FROM DrugAlone, SideEffect
WHERE DrugAlone.idPerson = SideEffect.idPerson
GROUP BY DrugAlone.codeDrug;

CREATE VIEW NumberOfUsesByDrug AS
SELECT codeDrug,count(DrugAlone.codeDrug) as uses
FROM DrugAlone
GROUP BY codeDrug;

CREATE VIEW AloneSideEffectFrequency AS
SELECT NumberOfUsesByDrug.codeDrug, CAST(NumberOfUsesByDrug.uses as DOUBLE)/ NumberOfSideEffectsByDrug.sideeffects as sideeffectfreq
FROM NumberOfSideEffectsByDrug,NumberOfUsesByDrug
WHERE NumberOfUsesByDrug.codeDrug = NumberOfSideEffectsByDrug.codeDrug

In [44]:
CREATE VIEW NumberOfSideEffectsByDrugCombination AS
SELECT druga,drugb,count(*) as sideeffects
FROM DrugIntersection, SideEffect
WHERE DrugIntersection.idPerson = SideEffect.idPerson
GROUP BY druga,drugb;


CREATE VIEW NumberOfUsesByDrugCombination AS
SELECT druga,drugb,count(*) as uses
FROM DrugIntersection
GROUP BY druga,drugb;

CREATE VIEW CombinedSideEffectFrequency AS
SELECT SideComb.druga, SideComb.drugb, CAST(NUses.uses AS DOUBLE)/SideComb.sideeffects as sideeffectfreq
FROM NumberOfSideEffectsByDrugCombination AS SideComb, NumberOfUsesByDrugCombination as NUses
WHERE SideComb.druga = NUses.druga and SideComb.drugb = NUSes.drugb

In [47]:
SELECT Comb.druga, Comb.drugb, Comb.sideeffectfreq/Alone.sideeffectfreq
FROM CombinedSideEffectFrequency as Comb, AloneSideEffectFrequency as Alone
WHERE Comb.druga = Alone.codeDrug;